In [1]:
using DataFrames, LinearAlgebra, Distributions, Revise, Serialization

In [2]:
string(@__DIR__) in LOAD_PATH || push!(LOAD_PATH, @__DIR__)
using BLPmodule; const m = BLPmodule;

┌ Info: Precompiling BLPmodule [top-level]
└ @ Base loading.jl:1317


In [3]:
# TODO: add σ to every call of compute_deltas

---
#### Mean utilities

In [4]:
df = deserialize("jls/df.jls");
firm_IDs_long = df.j;
tract_IDs_long = df.t;
X = Matrix{Float64}(df[!, [:x1, :x2]])
D = Matrix{Float64}(df[!, [:d, :d2]])
Q = df.q
M = df.M;
nJ = length(unique(firm_IDs_long));
nI = 100;

In [22]:
σ = ones(2)
pars = m.set_Pars(K = 2, nI = nI, δs=ones(nJ,1));
serialize("jls/pars.jls", pars)

---

In [24]:
ec = m.make_Economy(
    firm_IDs_long,
    tract_IDs_long,
    X,
    D,
    Q,
    M,
    nI
    );


Economy with 394 firms and 3110 tracts.


In [25]:
serialize("jls/ec.jls", ec)

---

In [7]:
Threads.nthreads()

16

In [10]:
σ = ones(2);

2-element Vector{Float64}:
 1.0
 1.0

In [11]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
deltas, q_iter = m.compute_deltas(ec, pars, σ,max_iter=1000, verbose=true);
# serialize("jls/deltas.jls", deltas);

dist = 9.672361458434864e-7, iterations = 276


In [12]:
using BenchmarkTools

In [19]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
it = 1
@time m.compute_deltas(ec, pars, σ, max_iter=it, verbose=false)

  0.014284 seconds (84.28 k allocations: 70.733 MiB)


([-3.2583435674716226; -4.049455917060103; … ; -2.6695319632056913; -2.978603171037032], [425.02011354845587; 595.0509200412007; … ; 448.8966485977807; 484.16710838265783])

In [20]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
it = 10
@time m.compute_deltas(ec, pars, σ, max_iter=it, verbose=false)

  0.053707 seconds (449.03 k allocations: 270.217 MiB)


([-3.9123195137815956; -5.06557178480748; … ; -3.9409845626170785; -4.025023884975683], [80.00000074786205; 80.9999946257318; … ; 86.00010108254946; 92.00006007946708])

In [27]:
ec = deserialize("jls/ec.jls");
pars = deserialize("jls/pars.jls");
it = 1000
@time m.compute_deltas(ec, pars, σ, max_iter=it, verbose=false)

  2.824389 seconds (11.19 M allocations: 6.000 GiB, 43.44% gc time)


([-3.9122188427217397; -5.0655245711271455; … ; -3.9408047510837516; -4.024905795553984], [80.0; 80.99999999990256; … ; 86.0; 92.00000000000003])

In [ ]:
# # this was the line that the memory profiler said was creating the vast majority of allocations
# # ...which doesn't make sense because it's not even in the inner loop
# # ...and it's the inner loop that's driving the allocations and times (according to benchmarks)
# @time for t in ec.tracts
#     t.abδ .= t.D * pars.nlcoefs
# end

___
continued in `gmm.ipynb`